In [ ]:
#Extract frames from video

import cv2

cap = cv2.VideoCapture("Video.mp4")

frames = []
if cap is not None:
  _,frame = cap.read()
  frames.append(frame)

In [ ]:
#Motion detection using background substraction
import cv2

back_sub = cv2.bgsegm.createBackgroundSubtractorMOG();

for frame in frames:
  fg = back_sub.apply(frame)

  contour, _ = cv2.findContours(fg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

  if cv2.contourArea(contour)>500:
    x,y,w,h = cv2.boundingRect(contour)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
  cv2.imshow("Motion", frame)
  cv2.waitKey()
  cv2.destroyAllWindows()

In [3]:
#Apply Segmentation on frame using pytorch model

import torch
from torchvision import models, transforms
import numpy as np
from google.colab.patches import cv2_imshow

# Load image and predict segmentation using deeplabv3 model
def segmentation_frame(image_path):

  frame = cv2.imread(image_path)

  model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()

  preprocess = transforms.Compose([transforms.ToPILImage(),
                                   transforms.ToTensor(),
                                   transforms.Resize((512,512)),
                                   transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])])
  input_tensor = preprocess(frame)
  input_batch = input_tensor.unsqueeze(0)

  output = model(input_batch)['out'][0]
  output_pred = output.argmax(0).cpu().numpy()

  original_size = (frame.shape[1],frame.shape[0])
  resized_segmentation = cv2.resize(output_pred,original_size,interpolation=cv2.INTER_NEAREST)

  return frame, resized_segmentation

# Colors for ovelay
def colorize_segmentation(segmentation_map):
  #This is array of RGB value to display mask on object
  color_map = np.array([0,0,0],
                       [128,0,0],
                       [0,128,0],
                       [0,0,128],
                       [128,128,0],
                       [0,128,128],
                       [128,128,128],
                       [64,0,0],
                       [0,64,0],
                       [0,0,64],
                       [64,64,0],
                       [0,64,64],
                       [128,0,0],
                       [0,128,0],
                       [0,0,128],
                       [64,64,64],
                       [128,128,0],
                       [0,128,128],
                       [95,0,0],
                       [0,95,128],)
  if np.max(segmentation_map)>=color_map[0]:
    raise ValueError("Segmentation value error")

  color_segmentation = color_map[segmentation_map]

  return color_segmentation

# Apply overlay on image
def overlay_image(original_image,color_seg,alpha=0.5):
  original_float = original_image.astype(float)
  color_segmentation_float = color_seg.astype(float)

  blend = cv2.addWeighted(original_float,alpha,color_segmentation_float,1-alpha)
  return blend.astype('uint8')


oringinal_image,resize_seg =segmentation_frame("Image.jpg")
color_seg =  colorize_segmentation(resize_seg)
blended_image = overlay_image(oringinal_image,color_seg,0.5)

cv2_imshow(blended_image)